# Baseline Popular Recommender
As with any ML project, we start with a non-ML approach as our baseline. It's very common in RecSys project to have popular recommender as not only a benchmark but also an actual component (retrieval stage).

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "001-baseline-popular"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-25 02:30:16.903 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 001-baseline-popular...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "001-baseline-popular",
  "notebook_persist_dp": "/home/jupyter/frostmourne/reco-algo/notebooks/data/001-baseline-popular",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")

# Implement

In [5]:
popular_items_df = (
    train_df.groupby(args.item_col, as_index=False)
    .size()
    .assign(
        score=lambda df: df["size"] / df["size"].max(),
        rec_ranking=lambda df: df["score"]
        .rank(method="first", ascending=False)
        .astype(int),
    )
    .sort_values(["rec_ranking"], ascending=[True])
)

top_popular_items_df = popular_items_df.head(args.top_K)

top_popular_items_df

,parent_asin,size,score,rec_ranking
2143,B0086VPUHI,1526,1.000000,1
3554,B01N3ASPNV,1495,0.979685,2
4337,B07YBXFDYN,1412,0.925295,3
2368,B00BGA9WK2,1238,0.811271,4
1775,B004RMK5QG,1102,0.722149,5
...,...,...,...,...
2789,B00L59D9HG,342,0.224115,96
933,B001EYUQDW,340,0.222805,97
2776,B00KWOSQC8,338,0.221494,98
3410,B01GY3601E,336,0.220183,99


In [6]:
recommendations_df = (
    val_df[[args.user_col]]
    .drop_duplicates()
    .assign(tmp=1)
    .pipe(
        lambda df: pd.merge(
            df, top_popular_items_df.assign(tmp=1), on="tmp", how="left"
        )
    )[[args.user_col, args.item_col, "score", "rec_ranking"]]
)

recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B0086VPUHI,1.000000,1
1,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B01N3ASPNV,0.979685,2
2,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B07YBXFDYN,0.925295,3
3,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00BGA9WK2,0.811271,4
4,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B004RMK5QG,0.722149,5
...,...,...,...,...
70795,AGQXUPHVOU7PHT6SVFMHWG7A3F3Q,B00L59D9HG,0.224115,96
70796,AGQXUPHVOU7PHT6SVFMHWG7A3F3Q,B001EYUQDW,0.222805,97
70797,AGQXUPHVOU7PHT6SVFMHWG7A3F3Q,B00KWOSQC8,0.221494,98
70798,AGQXUPHVOU7PHT6SVFMHWG7A3F3Q,B01GY3601E,0.220183,99


# Evaluate

## Ranking metrics

In [7]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
303,AEMA3SW3WPNLEH3IACW23K2ZSUFA,B09JDLC31H,4.0,1.0
1339,AEURBISVS35ALE7YQLR5L4K7AHCA,B07QQ8N7LL,1.0,1.0
1648,AFX6IXYCJCVUAYTSPTCXHJXY3ZTQ,B076B995TL,5.0,1.0
1286,AGMBK7EWZQWYI74JQMATQGZZRFUQ,B0BL9CD53Z,4.0,1.0
1727,AEOY2365QPPEVDTOXL6N7ZA4NSAA,B00PDRZG9U,5.0,1.0
...,...,...,...,...
301,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B006QRNKOO,0.0,18.0
1832,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B017C6OK7S,0.0,18.0
1875,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00K0NV5J2,0.0,18.0
1501,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00308JV6I,0.0,19.0


In [8]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
26,AE2AZ2MNROPF33U6SS53VI22OXJA,B0086VPUHI,1.000000,1,0,NaN
81,AE2AZ2MNROPF33U6SS53VI22OXJA,B01N3ASPNV,0.979685,2,0,NaN
90,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YBXFDYN,0.925295,3,0,NaN
32,AE2AZ2MNROPF33U6SS53VI22OXJA,B00BGA9WK2,0.811271,4,0,NaN
19,AE2AZ2MNROPF33U6SS53VI22OXJA,B004RMK5QG,0.722149,5,0,NaN
...,...,...,...,...,...,...
72422,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00KWOSQC8,0.221494,98,0,NaN
72439,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B01GY3601E,0.220183,99,0,NaN
72406,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00DB2BI8M,0.218873,100,0,NaN
72445,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B07CMKDB2K,NaN,101,3,1.0


In [9]:
ranking_report = log_ranking_metrics(args, eval_df)

## Classification metrics

In [10]:
eval_classification_df = pd.merge(
    val_df,
    popular_items_df[[args.item_col, "score"]],
    on=[args.item_col],
    how="left",
    validate="m:1",
).assign(label=lambda df: df[args.rating_col].gt(0).astype(int))
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,score,label
0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B001F8188Y,0.0,1640980885681,5473,2668,Video Games,Nintendo DS Lite Cobalt / Black,"[Product Description, Nintendo DS Lite Cobalt ...","[Video Games, Legacy Systems, Nintendo Systems...",117.99,"[-1, -1, 3287, 169, 2617, 2413, 3088, 3283, 28...",0.173657,0
1,AHH7QSUK7NCWVYXT4DGC3WD2QMAQ,B07GP2CV85,0.0,1646754495061,8164,2501,Video Games,Metro Exodus: Day One Edition - PlayStation 4,"[Metro Exodus is an epic, story driven first p...","[Video Games, PlayStation 4, Games]",39.99,"[3105, 2926, 4020, 1750, 880, 2559, 689, 4467,...",0.049803,0
2,AGMBK7EWZQWYI74JQMATQGZZRFUQ,B0012N7BSI,0.0,1628828874493,4941,3774,Video Games,Madden NFL 09 - Playstation 3,"[Product description, Madden NFL 09 delivers i...","[Video Games, Legacy Systems, PlayStation Syst...",21.06,"[-1, -1, -1, 2094, 2112, 161, 1649, 1920, 1729...",0.020970,0
3,AF2L5SZ5P34FX44NEOYKLMY6WSTA,B0080CAOYM,0.0,1629274141747,4741,2067,Video Games,Call of Duty: Black Ops 2 [Download],"[Zombies, The Biggest Zombies Experience to Da...","[Video Games, PC, Games]",None,"[-1, -1, -1, -1, 612, 2050, 2080, 3583, 785, 201]",0.193316,0
4,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,17628,297,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[2186, 3468, 4604, 4223, 2586, 2262, 1549, 199...",0.108781,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B08N6NCR3Q,4.0,1642699950266,16864,76,Video Games,Thrustmaster T 16000M SPACE SIM DUO STICK (PC),[The THRUSTMASTER T.16000M FCS Space Sim Duo c...,"[Video Games, PC, Accessories, Controllers, Fl...",119.51,"[-1, -1, -1, -1, 4239, 2038, 3617, 4672, 2949,...",0.041940,1
1894,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,6281,2502,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 4391, 2658, 2729, 3312, 3222, 3759, 1892,...",0.068152,1
1895,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,8504,4485,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 4395, 2664, 678, 1806, 4185]",0.079948,1
1896,AHUA2BO4FFD3VLGEOIF4TGJYCOBQ,B01GY35GIM,0.0,1647432520927,16980,758,Video Games,Titanfall 2 - SteelBook Edition - PlayStation 4,"[I bought it about 2 weeks ago, now I have two...","[Video Games, PlayStation 4, Games]",None,"[2375, 1501, 4639, 2856, 4680, 4401, 950, 4074...",0.330931,0


In [11]:
eval_classification_df["score"].describe().T

count    1898.000000
mean        0.111257
std         0.152932
min         0.009174
25%         0.028178
50%         0.055701
75%         0.121232
max         1.000000
Name: score, dtype: float64

In [12]:
classification_report = log_classification_metrics(
    args, eval_classification_df, target_col="label", prediction_col="score"
)

## Loss
We can estimate what kinds of MSE loss should be the upper bound model training based on measuring the loss when asking the model to naively predict the mean rating for every item.

In [13]:
naive_prediction = train_df[args.rating_col].mean()

naive_mse = (
    (
        val_df[args.rating_col]
        - val_df.assign(naive_prediction=naive_prediction)["naive_prediction"]
    )
    .apply(lambda s: s * s)
    .mean()
)
logger.info(
    f"Val MSE = {naive_mse:,.2f} given naive_prediction={naive_prediction:,.1f}"
)

2024-09-25 02:30:24.789 | INFO     | __main__:<module>:11 - Val MSE = 5.12 given naive_prediction=2.2


# Clean up

In [14]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/25 02:30:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run 001-baseline-popular at: http://localhost:5003/#/experiments/2/runs/49efcfb2716d4d968a7f8e7fc2c5e249.
2024/09/25 02:30:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/2.
